In [1]:
import requests
import json
import os
from dotenv import load_dotenv

## Config Meta App + Instagram Variables

In [2]:
load_dotenv()

ig_user_id = os.getenv("IG_USER_ID")
app_id = os.getenv("APP_ID")
app_secret = os.getenv("APP_SECRET")
user_access_token = os.getenv("USER_ACCESS_TOKEN")

In [ ]:
# https://graph.facebook.com/v19.0/oauth/access_token
# ?grant_type=fb_exchange_token
# &client_id={app_id}
# &client_secret={app_secret}
# &fb_exchange_token={user_access_token}

## Get Long Access Token

In [3]:
url = f"https://graph.facebook.com/v19.0/oauth/access_token?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={user_access_token}"
response = requests.get(url)
#print(response.content)
long_access_token = response.json()["access_token"]
long_access_token

'EAAYMUDcHYToBPFD5XHPlsIFNMapl4UrnUv063KlqzA6B1VSuLZAe80QzSZCaxL9BUbiKqJ7dq0KBcco4m0Tm7xEckFZB9ztdwLvKawxm8vz7LrOAdL3Dm3r0oT4yUvzxEbXnv7vL2xhCCYktMsQhXjnmuE7OAmbMUmeFfm3BfgLZC7MaoSpFZBKq5Quv2'

## Test API

In [ ]:
"https://graph.facebook.com/v23.0/17841405309211844 \
  ?fields=business_discovery.username(bluebottle){followers_count,media_count} \
  &access_token=<YOUR_APP_USERS_INSTAGRAM_USER_ACCESS_TOKEN>"

In [ ]:
# Get account details
username="jestestbusiness"
required_param = "{followers_count,media_count}"
url = f"https://graph.facebook.com/v19.0/{ig_user_id}?fields=business_discovery.username({username}){required_param}&access_token={long_access_token}"
response = requests.get(url)
metadata= response.json()
followers_count = metadata["business_discovery"]["followers_count"]
media_count = metadata["business_discovery"]["media_count"]
print(json.dumps(metadata, indent=4))

### Check User Page (Does not work on Private Accounts)

In [8]:
ig_username = "theforgottenwan_"
required_param = "{name,website,biography,followers_count,follows_count,media_count,profile_picture_url,media.limit(5){thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink}}"

In [9]:
url = f"https://graph.facebook.com/v19.0/{ig_user_id}?fields=business_discovery.username({ig_username}){required_param}&access_token={long_access_token}"
response = requests.get(url)
metadata = response.json()
print(json.dumps(metadata, indent=4))

{
    "business_discovery": {
        "name": "ImJustNobody",
        "followers_count": 2,
        "follows_count": 708,
        "media_count": 0,
        "profile_picture_url": "https://scontent.fmnl13-4.fna.fbcdn.net/v/t51.2885-15/412496631_749883563827902_8239433423865633342_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=7d201b&_nc_ohc=QWsq7UrPwCUQ7kNvwHj0jnj&_nc_oc=Adl1S5HBHXNc71lshEXa1ZVPQBZP9Pk1iJqNZDOBgaQCJKauBTqdxsvWG_fs_J2ZOC6tAYmL2ooyITojXHTsz839&_nc_zt=23&_nc_ht=scontent.fmnl13-4.fna&edm=AL-3X8kEAAAA&oh=00_AfSTIOBhp_ay7ojn5KM88neV26CvhSuQ5V2ZSXgfXuyBQA&oe=689268A5",
        "id": "17841414524839587"
    },
    "id": "17841476287277831"
}


### Download a Post and Thumbnail

In [ ]:
first_post_link = metadata["business_discovery"]["media"]["data"][0]["media_url"]
print(f"Media URL : {first_post_link} \n")
first_post_thumbnail_link = metadata["business_discovery"]["media"]["data"][0]["thumbnail_url"]
print(f"Media Thumbnail URL : {first_post_thumbnail_link}")

In [ ]:
os.makedirs("./temp", exist_ok=True)

response = requests.get(first_post_link)
with open ("./temp/first_post.mp4", "wb") as f:
    f.write(response.content)

In [ ]:
response = requests.get(first_post_thumbnail_link)
with open ("./temp/first_post_thumbnail_link.jpg", "wb") as f:
    f.write(response.content)

### Pagination

In [ ]:
{media.limit(1).after(your_after_code){your_fields_by_comma_separation}}

In [ ]:
after_code = metadata["business_discovery"]["media"]["paging"]["cursors"]["after"]
after_code

In [ ]:
required_param = "{media.after(" + after_code + "){thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink}}"
url = f"https://graph.facebook.com/v19.0/{ig_user_id}?fields=business_discovery.username({ig_username}){required_param}&access_token={long_access_token}"
response = requests.get(url)
metadata = response.json()
print(json.dumps(metadata, indent=4))

## Manage comments and Auto-Reply

In [ ]:
#https://developers.facebook.com/docs/instagram-platform/instagram-api-with-facebook-login/comment-moderation
#https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-media/comments
#https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-comment/replies#create
#https://developers.facebook.com/docs/instagram-platform/reference/instagram-media#edges
#https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media#reading

### Get Comments & Replies

In [4]:
business_discovery_parameters = "thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink"
other_parameters = "comments{id,from,text,timestamp,like_count,media,parent_id,replies{from,timestamp,like_count,text},hidden,user,username}"
fields = business_discovery_parameters + "," + other_parameters
fields

'thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink,comments{id,from,text,timestamp,like_count,media,parent_id,replies{from,timestamp,like_count,text},hidden,user,username}'

In [5]:
url = f"https://graph.facebook.com/v23.0/{ig_user_id}/media"
payload = {
    "fields": fields,
    "access_token": long_access_token
}
response = requests.get(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))

{
    "data": [
        {
            "media_type": "IMAGE",
            "media_product_type": "FEED",
            "timestamp": "2025-07-31T06:43:25+0000",
            "like_count": 1,
            "comments_count": 2,
            "media_url": "https://scontent.cdninstagram.com/v/t51.82787-15/526631522_17843275647542910_4921033368664523029_n.jpg?stp=dst-jpg_e35_tt6&_nc_cat=101&ccb=1-7&_nc_sid=18de74&_nc_ohc=cNGh4eKOu5YQ7kNvwG2J7Lg&_nc_oc=AdmV7B-QrzzJhnO_gp9V8uMQjVHWSNCWUnZObUhl2B5CBaqWtaZQhlK05bKZ7JkV6zEBAatYI6fwx9aXk8wfmN7q&_nc_zt=23&_nc_ht=scontent.cdninstagram.com&edm=AM6HXa8EAAAA&_nc_gid=HHvPOuXukL4Ox56pUR9vwA&oh=00_AfRzL1rIwWXDmcuRkA7C0K1tRJnBnissI4HuR3Q7VyxFww&oe=6892562B",
            "permalink": "https://www.instagram.com/p/DMwzUlFTU2H/",
            "comments": {
                "data": [
                    {
                        "id": "18047345528249440",
                        "from": {
                            "id": "655555347550295",
                            "us

In [ ]:
image_comment = data["data"][0]["comments"]["data"]
print(json.dumps(image_comment, indent=4))

### Add Comments

In [ ]:
media_id = data["data"][0]["id"]
media_id

In [ ]:
url = f"https://graph.facebook.com/v23.0/{media_id}/comments"
payload = {
    "message": "PM US for more info!",
    "access_token": long_access_token
}
response = requests.post(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))

### Reply to comments

In [ ]:
comment_id = data["data"][0]["comments"]["data"][1]["id"]
comment_id

In [ ]:
url = f"https://graph.facebook.com/v23.0/{comment_id}/replies"
payload = {
    "message": "Bonjour",
    "access_token": long_access_token
}
response = requests.post(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))

### Hide | Unhide Comments

In [ ]:
comment_id = 18074775181988435

In [ ]:
url = f"https://graph.facebook.com/v23.0/{comment_id}/"
payload = {
    "hide": "false",
    "access_token": long_access_token
}
response = requests.post(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))

### Disable | Enable Comments

In [ ]:
media_id = 18171023602348133

In [ ]:
url = f"https://graph.facebook.com/v23.0/{media_id}/"
payload = {
    "comment_enabled": "true",
    "access_token": long_access_token
}
response = requests.post(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))

### Delete Comments

In [ ]:
comment_id = 18321019534236920

In [ ]:
url = f"https://graph.facebook.com/v23.0/{comment_id}/"
payload = {
    "access_token": long_access_token
}
response = requests.delete(url, params=payload)
data = response.json()
print(json.dumps(data, indent=4))